# EI ST4

## Imports

In [ ]:
import pandas as pd
from datetime import datetime
import math

In [ ]:
df = pd.read_csv(r"DS_CentraleSupelec_ST42021/DS_CentraleSupelec_train.csv")

## Cleaning up the dataframe

In [ ]:
df

Get unique count for each variable

In [ ]:
df.nunique()

Check variable data type

In [ ]:
df.dtypes

We can remove the `languageCode` and `countryCode` column as they are constant

In [ ]:
df[~df.languageCode.isin(['fr', 'FR'])].empty and df[~df.countryCode.eq('FRA')].empty

In [ ]:
df = df.drop(columns=['languageCode', 'countryCode'])

We will also remove the `registrationDate`, `reactivationValue`, `emailContactable` and `postalContactable` as they are irrelevant

In [ ]:
df = df.drop(columns=['registrationDate', 'reactivationValue', 'emailContactable', 'postalContactable'])

We are going to replace the `washDishes` `STRING` column with a `ownsDishwasher` `INT` column

In [ ]:
df['ownsDishwasher'] = df.apply(lambda row: 1 if row['washDishes'] == 'Auto' else -1, axis=1)
df = df.drop(columns=["washDishes"])

In the `MrPropreTrier`, `AntikalTrier`, `ArielTrier`, `DashTrier` we will replace `Known Trier` with `1` and `Nan` with `-1`

In [ ]:
for c in ['MrPropreTrier', 'AntikalTrier', 'ArielTrier', 'DashTrier']:
    df[c[:-5]] = df.apply(lambda row: 1 if row[c] == 'Known Trier' else -1, axis=1)
    df = df.drop(columns=[c])

We will replace `detergentType` by `liquidDetergent`

In [ ]:
df['liquidDetergent'] = df.apply(lambda row: 1 if row['detergentType'] == 'Liquid' else -1, axis=1)
df = df.drop(columns=['detergentType'])

Replacing `toothBrushType` with `electricToothbrush`

In [ ]:
df['electricToothbrush'] = df.apply(lambda row: 1 if row['toothBrushType'] == 'Electric' else -1, axis=1)
df = df.drop(columns=['toothBrushType'])

Replacing `petOwner` with `hasPet`, if the first 3 characters are 'Yes' or 'Oui' the value is `1`, else it is `-1`

In [ ]:
df['hasPet'] = df.apply(lambda row: 1 if row['petOwner'][:3] == 'Yes' or row['petOwner'][:3] == 'Oui' else -1, axis=1)
df = df.drop(columns=['petOwner'])

Transforms a `ISO 8601` time string to the amount of days since the time string

In [ ]:
def timeStringToDelta(timeString: str) -> int:
    return int((datetime.utcnow().timestamp() - datetime.strptime(timeString[:10], "%Y-%m-%d").timestamp()) / 3600 / 24)

Replaces `lastActivityDate` with `daysSinceActivity`: an `integer` representing the amount of ellapsed days since last activity

In [ ]:
df['daysSinceActivity'] = df.apply(lambda row: timeStringToDelta(row['lastActivityDate']), axis=1)
df = df.drop(columns=['lastActivityDate'])

In [ ]:
df

The possible values for `numberChildren` are `'1'`, `'2'`, `'3'`, `'4'`, `'4+'` and `'NaN'`. We will be creating a column `nbChildren` of type `int` where `'NaN'` will be mapped to `None`.

In [ ]:
def childrenMagik(children: str):
    if str(children).lower() in 'nan': return None
    if children == '4+': return 5
    return int(children)

df['nbChildren'] = df.apply(lambda row: childrenMagik(row['numberChildren']), axis=1)
df = df.drop(columns=['numberChildren'])

In [ ]:
df

In [ ]:
df['nbChildren'].mean()

In [ ]:
def textScaleToFloat(s:str) -> float:
    if s == 'High': return 1
    if s == 'Medium': return 2 / 3
    if s == 'Low': return 1 / 3
    if s == 'NonUser': return 0
    return None

# replaces the 'normal' values with floats
df['scentLover'] = df['scentLover'].apply(textScaleToFloat)

# replaces the nan with the avg
df['scentLover'] = df['scentLover'].fillna(df['scentLover'].mean(skipna=True))

Handles the closest shop

In [ ]:
df['magasin']           = df['closestShop'].apply(lambda e: 1 if e =='1. Magasin de Proximit�' else -1)
df['moyenneSurgace']    = df['closestShop'].apply(lambda e: 1 if e =='2. Moyenne Surface' else -1)
df['superMarket']       = df['closestShop'].apply(lambda e: 1 if e =='3. SuperMarket' else -1)
df['hyperMarket']       = df['closestShop'].apply(lambda e: 1 if e =='4. HyperMarket' else -1)
df['drive']             = df['closestShop'].apply(lambda e: 1 if e =='5. Drive' else -1)
df['hardDiscount']      = df['closestShop'].apply(lambda e: 1 if e =='6. Hard Discounter' else -1)

# removes extra column
df.drop(columns=['closestShop'])

In [ ]:
df

In order to have more information about customers, we are going to add features, using the data set "consumer actions"


In [ ]:
df_consumer_actions = pd.read_csv(r"DS_CentraleSupelec_ST42021/DS_CentraleSupelec_consumeractions.csv")

In [ ]:
df_consumer_actions = pd.read_csv(r"DS_CentraleSupelec_ST42021/DS_CentraleSupelec_consumeractions.csv")
df_consumer_actions

text = df_consumer_actions['brandName'][0]
df_consumer_actions['brandName'] = df_consumer_actions['brandName']== text
df_consumer_actions['brandName2'] = df_consumer_actions['brandName2']== text
df_consumer_actions['brand'] = df_consumer_actions['brandName'] | df_consumer_actions['brandName2']
df_consumer_actions =  df_consumer_actions.drop(columns=['event', 'originalTimestamp', 'brandName', 'brandName2', 'subjectLine', 'url', 'keyword','couponRedemptionDate'])
df_consumer_actions['brand'] = df_consumer_actions.apply(lambda row: 1 if row['brand'] else -1, axis=1)
df_consumer_actions

df_merged = pd.merge(df,df_consumer_actions, how='left')

df_merged = df_merged.rename(columns={"brand": "Interested_email"})
df = df_merged

In [ ]:

text = df_consumer_actions['brandName'][0]
df_consumer_actions['brandName'] = df_consumer_actions['brandName']== text
df_consumer_actions['brandName2'] = df_consumer_actions['brandName2']== text
df_consumer_actions['brand'] = df_consumer_actions['brandName'] | df_consumer_actions['brandName2']
df_consumer_actions =  df_consumer_actions.drop(columns=['event', 'originalTimestamp', 'brandName', 'brandName2', 'subjectLine', 'url', 'keyword','couponRedemptionDate'])
df_consumer_actions['brand'] = df_consumer_actions.apply(lambda row: 1 if row['brand'] else -1, axis=1)
df_consumer_actions

In [ ]:
df_merged = pd.merge(df,df_consumer_actions, how='left')


In [ ]:
df_merged = df_merged.rename(columns={"brand": "Interested_email"})
df = df_merged

In [ ]:
df